# Cambridge Bike Crashes Analysis

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from folium import plugins
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


Libraries imported successfully!
Pandas version: 2.3.2
NumPy version: 2.3.3


## 1. Data Loading and Initial Exploration


In [2]:
# Load the crash data
data_path = './data/Police_Department_Crash_Data_-_Updated_20250928.csv'
df = pd.read_csv(data_path)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display basic info
print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)
print(f"Total records: {len(df):,}")
print(f"Date range: {df['Date Time'].min()} to {df['Date Time'].max()}")
print(f"Unique streets: {df['Street Name'].nunique():,}")
print(f"Unique intersections: {df['Street or Intersection'].nunique():,}")


Dataset loaded successfully!
Shape: (15435, 159)
Columns: 159
Memory usage: 131.55 MB

DATASET OVERVIEW
Total records: 15,435
Date range: 01/01/2015 01:12:00 AM to 12/31/2024 08:54:00 AM
Unique streets: 2,144
Unique intersections: 2


In [3]:
# Examine the structure of the data
print("COLUMN NAMES AND TYPES:")
print("="*50)
print(df.dtypes)

print("\n\nFIRST FEW ROWS:")
print("="*50)
print(df.head())

print("\n\nSAMPLE OF KEY COLUMNS:")
print("="*50)
key_columns = ['Date Time', 'Day of Week', 'Street Name', 'Cross Street', 
               'May involve cyclist', 'May Involve Pedestrian', 'Manner of Collision',
               'Weather Condition 1', 'Traffic Control Device Type', 'Location']
print(df[key_columns].head(10))


COLUMN NAMES AND TYPES:
Date Time           object
Day of Week         object
Object 1            object
Object 2            object
Street Number       object
                     ...  
P2 Sex              object
P2 Seat Position    object
P2 Safety System    object
P2 Trapped          object
P2 Veh Owner        object
Length: 159, dtype: object


FIRST FEW ROWS:
                Date Time Day of Week  \
0  08/31/2025 07:25:00 PM      Sunday   
1  08/31/2025 07:57:00 AM      Sunday   
2  08/31/2025 07:12:00 AM      Sunday   
3  08/30/2025 10:00:00 PM    Saturday   
4  08/30/2025 06:50:00 PM    Saturday   

                                            Object 1  \
0  PASSENGER CAR                                 ...   
1  OTHER                                         ...   
2  PASSENGER CAR                                 ...   
3  PASSENGER CAR                                 ...   
4  PASSENGER CAR                                 ...   

                                            Object

In [4]:
# Check for missing values
print("MISSING VALUES ANALYSIS:")
print("="*50)
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing Count': missing_data.values,
    'Missing Percentage': missing_percent.values
}).sort_values('Missing Percentage', ascending=False)

# Show columns with missing data
print("Columns with missing values:")
print(missing_df[missing_df['Missing Count'] > 0].head(20))

print(f"\nTotal columns with missing data: {len(missing_df[missing_df['Missing Count'] > 0])}")
print(f"Columns with >50% missing: {len(missing_df[missing_df['Missing Percentage'] > 50])}")
print(f"Columns with >90% missing: {len(missing_df[missing_df['Missing Percentage'] > 90])}")


MISSING VALUES ANALYSIS:
Columns with missing values:
                    Column  Missing Count  Missing Percentage
122           V2 Is Hazmat          15435          100.000000
97   V2 Underride Override          15435          100.000000
55   V1 Underride Override          15435          100.000000
80            V1 Is Hazmat          15434           99.993521
111         V2 Haz Release          15433           99.987042
69          V1 Haz Release          15432           99.980564
110         V2 Haz Placard          15416           99.876903
47                V1 Moped          15402           99.786200
116         V2 Trailer Len          15396           99.747328
68          V1 Haz Placard          15394           99.734370
107          V2 Interstate          15394           99.734370
115    V2 Trailer Reg Year          15392           99.721412
112    V2 Trailer Reg Type          15380           99.643667
120         V2 Has Trailer          15378           99.630709
123            V

## 2. Data Cleaning and Street Name Standardization


In [5]:
# First, let's analyze the bicycle crashes and see the street name issues
print("BICYCLE CRASHES ANALYSIS:")
print("="*50)

# Filter bicycle-related crashes
bike_indicators = (
    (df['May involve cyclist'] == '1') |
    (df['Object 1'].str.contains('BICYCLE|PEDALCYCLE', case=False, na=False)) |
    (df['Object 2'].str.contains('BICYCLE|PEDALCYCLE', case=False, na=False)) |
    (df['First Harmful Event'].str.contains('PEDALCYCLE', case=False, na=False)) |
    (df['P1 Non Motorist Desc'].str.contains('BICYCLE|PEDALCYCLE', case=False, na=False)) |
    (df['P2 Non Motorist Desc'].str.contains('BICYCLE|PEDALCYCLE', case=False, na=False))
)

bike_crashes = df[bike_indicators].copy()
print(f"Total bicycle-related crashes: {len(bike_crashes):,}")

# Show the problematic street names
print("\nTop 15 streets with bicycle crashes (before cleaning):")
top_streets = bike_crashes['Street Name'].value_counts().head(15)
print(top_streets)

# Show the variety of street name formats
print(f"\nExamples of street name variations:")
mass_ave_variations = bike_crashes[bike_crashes['Street Name'].str.contains('MASS', case=False, na=False)]['Street Name'].unique()
print("Massachusetts Avenue variations:")
for var in mass_ave_variations:
    print(f"  - '{var}'")

cambridge_variations = bike_crashes[bike_crashes['Street Name'].str.contains('CAMBRIDGE', case=False, na=False)]['Street Name'].unique()
print("\nCambridge Street variations:")
for var in cambridge_variations:
    print(f"  - '{var}'")


BICYCLE CRASHES ANALYSIS:
Total bicycle-related crashes: 1,137

Top 15 streets with bicycle crashes (before cleaning):
Street Name
MASSACHUSETTS AVE                                     120
MASS AVE                                               96
MASSACHUSETTS AVENUE                                   84
BROADWAY                                               66
CAMBRIDGE ST                                           65
CAMBRIDGE STREET                                       41
HAMPSHIRE STREET                                       35
HAMPSHIRE ST                                           28
BRATTLE STREET                                         19
MAIN ST                                                14
MAIN STREET                                            12
GARDEN ST                                              12
PROSPECT STREET                                        12
VASSAR ST                                              11
MASSACHUSETTS AVE.                                     10

In [6]:
# Create comprehensive street name standardization function
def standardize_street_name(street_name):
    """
    Standardize street names to a consistent format
    """
    if pd.isna(street_name) or street_name == '':
        return None
    
    # Convert to uppercase and strip whitespace
    street = str(street_name).strip().upper()
    
    # Remove extra spaces and standardize common abbreviations
    street = ' '.join(street.split())
    
    # Common street name mappings
    street_mappings = {
        # Massachusetts Avenue variations
        'MASS AVE': 'MASSACHUSETTS AVENUE',
        'MASSACHUSETTS AVE': 'MASSACHUSETTS AVENUE',
        'MASSACHUSETTS AV': 'MASSACHUSETTS AVENUE',
        
        # Cambridge Street variations
        'CAMBRIDGE ST': 'CAMBRIDGE STREET',
        'CAMBRIDGE ST.': 'CAMBRIDGE STREET',
        
        # Hampshire Street variations
        'HAMPSHIRE ST': 'HAMPSHIRE STREET',
        'HAMPSHIRE ST.': 'HAMPSHIRE STREET',
        
        # Brattle Street variations
        'BRATTLE ST': 'BRATTLE STREET',
        'BRATTLE ST.': 'BRATTLE STREET',
        
        # Main Street variations
        'MAIN ST': 'MAIN STREET',
        'MAIN ST.': 'MAIN STREET',
        
        # Broadway variations (usually just BROADWAY)
        'BROADWAY ST': 'BROADWAY',
        'BROADWAY STREET': 'BROADWAY',
        
        # Prospect Street variations
        'PROSPECT ST': 'PROSPECT STREET',
        'PROSPECT ST.': 'PROSPECT STREET',
        
        # Common abbreviations
        'ST.': 'STREET',
        'AVE.': 'AVENUE',
        'AV.': 'AVENUE',
        'RD.': 'ROAD',
        'BLVD.': 'BOULEVARD',
        'PKWY.': 'PARKWAY',
        'DR.': 'DRIVE',
        'CT.': 'COURT',
        'PL.': 'PLACE',
        'LN.': 'LANE',
        'WAY.': 'WAY',
        'CIR.': 'CIRCLE',
        'TER.': 'TERRACE',
    }
    
    # Apply mappings
    for old, new in street_mappings.items():
        if street.endswith(' ' + old):
            street = street[:-len(old)-1] + ' ' + new
        elif street == old:
            street = new
    
    # Handle specific street name corrections
    specific_corrections = {
        'MASSACHUSETTS AVE': 'MASSACHUSETTS AVENUE',
        'CAMBRIDGE ST': 'CAMBRIDGE STREET',
        'HAMPSHIRE ST': 'HAMPSHIRE STREET',
        'BRATTLE ST': 'BRATTLE STREET',
        'MAIN ST': 'MAIN STREET',
        'PROSPECT ST': 'PROSPECT STREET',
        'OXFORD ST': 'OXFORD STREET',
        'WALDEN ST': 'WALDEN STREET',
        'PEARL ST': 'PEARL STREET',
        'ELLERY ST': 'ELLERY STREET',
        'LAMBERT ST': 'LAMBERT STREET',
        'SPRINGFIELD ST': 'SPRINGFIELD STREET',
        'FAWCETT ST': 'FAWCETT STREET',
        'SCIARAPPA ST': 'SCIARAPPA STREET',
        'WENDELL ST': 'WENDELL STREET',
        'WILLARD ST': 'WILLARD STREET',
        'SUMNER RD': 'SUMNER ROAD',
        'ALBANY ST': 'ALBANY STREET',
        'BEECH ST': 'BEECH STREET',
        'STATE ST': 'STATE STREET',
        'WATER ST': 'WATER STREET',
        'CARDINAL MEDEIROS AVENUE': 'CARDINAL MEDEIROS AVENUE',  # Keep as is
        'ALEWIFE BROOK PARKWAY': 'ALEWIFE BROOK PARKWAY',  # Keep as is
        'FRESHPOND PARKWAY': 'FRESHPOND PARKWAY',  # Keep as is
        'MONSIGNOR O\'BRIEN HIGHWAY': 'MONSIGNOR O\'BRIEN HIGHWAY',  # Keep as is
    }
    
    # Apply specific corrections
    if street in specific_corrections:
        street = specific_corrections[street]
    
    return street

# Test the standardization function
print("TESTING STREET NAME STANDARDIZATION:")
print("="*50)

test_streets = [
    'MASS AVE', 'MASSACHUSETTS AVE', 'MASSACHUSETTS AVENUE',
    'CAMBRIDGE ST', 'CAMBRIDGE STREET',
    'HAMPSHIRE ST', 'HAMPSHIRE STREET',
    'BROADWAY', 'BROADWAY ST',
    'MAIN ST', 'MAIN STREET'
]

for test_street in test_streets:
    standardized = standardize_street_name(test_street)
    print(f"'{test_street}' -> '{standardized}'")


TESTING STREET NAME STANDARDIZATION:
'MASS AVE' -> 'MASSACHUSETTS AVENUE'
'MASSACHUSETTS AVE' -> 'MASSACHUSETTS AVENUE'
'MASSACHUSETTS AVENUE' -> 'MASSACHUSETTS AVENUE'
'CAMBRIDGE ST' -> 'CAMBRIDGE STREET'
'CAMBRIDGE STREET' -> 'CAMBRIDGE STREET'
'HAMPSHIRE ST' -> 'HAMPSHIRE STREET'
'HAMPSHIRE STREET' -> 'HAMPSHIRE STREET'
'BROADWAY' -> 'BROADWAY'
'BROADWAY ST' -> 'BROADWAY'
'MAIN ST' -> 'MAIN STREET'
'MAIN STREET' -> 'MAIN STREET'


In [7]:
# Apply street name standardization to bicycle crashes
print("APPLYING STREET NAME STANDARDIZATION:")
print("="*50)

# Apply standardization to both Street Name and Cross Street
bike_crashes['Street Name Cleaned'] = bike_crashes['Street Name'].apply(standardize_street_name)
bike_crashes['Cross Street Cleaned'] = bike_crashes['Cross Street'].apply(standardize_street_name)

# Show results after cleaning
print("Top 15 streets with bicycle crashes (after cleaning):")
top_streets_cleaned = bike_crashes['Street Name Cleaned'].value_counts().head(15)
print(top_streets_cleaned)

# Show improvement
print(f"\nIMPROVEMENT SUMMARY:")
print(f"Before cleaning: {bike_crashes['Street Name'].nunique()} unique street names")
print(f"After cleaning: {bike_crashes['Street Name Cleaned'].nunique()} unique street names")
print(f"Reduction: {bike_crashes['Street Name'].nunique() - bike_crashes['Street Name Cleaned'].nunique()} duplicate names eliminated")

# Show specific improvements for major streets
print(f"\nSPECIFIC IMPROVEMENTS:")
mass_ave_before = bike_crashes[bike_crashes['Street Name'].str.contains('MASS', case=False, na=False)]['Street Name'].value_counts()
mass_ave_after = bike_crashes[bike_crashes['Street Name Cleaned'] == 'MASSACHUSETTS AVENUE']['Street Name Cleaned'].value_counts()

print(f"Massachusetts Avenue:")
print(f"  Before: {mass_ave_before.sum()} crashes across {len(mass_ave_before)} variations")
print(f"  After: {mass_ave_after.sum()} crashes unified under 'MASSACHUSETTS AVENUE'")

cambridge_before = bike_crashes[bike_crashes['Street Name'].str.contains('CAMBRIDGE', case=False, na=False)]['Street Name'].value_counts()
cambridge_after = bike_crashes[bike_crashes['Street Name Cleaned'] == 'CAMBRIDGE STREET']['Street Name Cleaned'].value_counts()

print(f"\nCambridge Street:")
print(f"  Before: {cambridge_before.sum()} crashes across {len(cambridge_before)} variations")
print(f"  After: {cambridge_after.sum()} crashes unified under 'CAMBRIDGE STREET'")


APPLYING STREET NAME STANDARDIZATION:
Top 15 streets with bicycle crashes (after cleaning):
Street Name Cleaned
MASSACHUSETTS AVENUE    312
CAMBRIDGE STREET        110
BROADWAY                 74
HAMPSHIRE STREET         70
MAIN STREET              28
BRATTLE STREET           26
PROSPECT STREET          23
OXFORD STREET            13
ALBANY STREET            12
JFK STREET               12
GARDEN ST                12
VASSAR ST                11
CONCORD AVE              10
COLUMBIA ST               9
GARDEN STREET             9
Name: count, dtype: int64

IMPROVEMENT SUMMARY:
Before cleaning: 265 unique street names
After cleaning: 236 unique street names
Reduction: 29 duplicate names eliminated

SPECIFIC IMPROVEMENTS:
Massachusetts Avenue:
  Before: 345 crashes across 20 variations
  After: 312 crashes unified under 'MASSACHUSETTS AVENUE'

Cambridge Street:
  Before: 115 crashes across 6 variations
  After: 110 crashes unified under 'CAMBRIDGE STREET'


In [8]:
# Extract coordinates from Location column
def extract_coordinates(location_str):
    """Extract latitude and longitude from location string"""
    if pd.isna(location_str):
        return None, None
    
    import re
    # Look for coordinates in parentheses format: (lat, lon)
    coord_pattern = r'\(([-+]?\d+\.?\d*),\s*([-+]?\d+\.?\d*)\)'
    match = re.search(coord_pattern, str(location_str))
    
    if match:
        try:
            lat = float(match.group(1))
            lon = float(match.group(2))
            return lat, lon
        except ValueError:
            return None, None
    return None, None

# Extract coordinates for bicycle crashes
bike_crashes['Latitude'] = None
bike_crashes['Longitude'] = None

for idx, row in bike_crashes.iterrows():
    lat, lon = extract_coordinates(row['Location'])
    bike_crashes.at[idx, 'Latitude'] = lat
    bike_crashes.at[idx, 'Longitude'] = lon

# Count crashes with coordinates
crashes_with_coords = bike_crashes.dropna(subset=['Latitude', 'Longitude'])
print(f"COORDINATE EXTRACTION RESULTS:")
print("="*40)
print(f"Total bicycle crashes: {len(bike_crashes):,}")
print(f"Crashes with coordinates: {len(crashes_with_coords):,}")
print(f"Percentage with coordinates: {len(crashes_with_coords)/len(bike_crashes)*100:.1f}%")

if len(crashes_with_coords) > 0:
    print(f"\nCoordinate statistics:")
    print(f"Latitude range: {crashes_with_coords['Latitude'].min():.4f} to {crashes_with_coords['Latitude'].max():.4f}")
    print(f"Longitude range: {crashes_with_coords['Longitude'].min():.4f} to {crashes_with_coords['Longitude'].max():.4f}")
    print(f"Center point: ({crashes_with_coords['Latitude'].mean():.4f}, {crashes_with_coords['Longitude'].mean():.4f})")


COORDINATE EXTRACTION RESULTS:
Total bicycle crashes: 1,137
Crashes with coordinates: 411
Percentage with coordinates: 36.1%

Coordinate statistics:
Latitude range: 42.3556 to 42.4004
Longitude range: -71.1555 to -71.0693
Center point: (42.3724, -71.1088)


In [9]:
# Create intersection identifiers and infer coordinates for missing ones
print("CREATING INTERSECTION IDENTIFIERS:")
print("="*40)

# Create intersection ID from cleaned street names
def create_intersection_id(street_name, cross_street):
    """Create a consistent intersection identifier"""
    if pd.isna(street_name) or pd.isna(cross_street):
        return None
    
    # Sort street names alphabetically for consistent intersection IDs
    streets = sorted([str(street_name).strip(), str(cross_street).strip()])
    return f"{streets[0]}_AND_{streets[1]}"

# Apply intersection ID creation
bike_crashes['Intersection_ID'] = bike_crashes.apply(
    lambda row: create_intersection_id(row['Street Name Cleaned'], row['Cross Street Cleaned']), 
    axis=1
)

# For crashes without cross streets, use just the street name
bike_crashes.loc[bike_crashes['Intersection_ID'].isna(), 'Intersection_ID'] = bike_crashes.loc[bike_crashes['Intersection_ID'].isna(), 'Street Name Cleaned']

# Infer coordinates for crashes without coordinates using intersection-based averaging
print("INFERRING COORDINATES FOR MISSING DATA:")
print("="*45)

# Group by intersection and calculate average coordinates
intersection_coords = bike_crashes.groupby('Intersection_ID').agg({
    'Latitude': 'mean',
    'Longitude': 'mean'
}).dropna()

print(f"Intersections with known coordinates: {len(intersection_coords)}")

# Fill missing coordinates using intersection averages
missing_coords_mask = bike_crashes['Latitude'].isna() | bike_crashes['Longitude'].isna()
crashes_missing_coords = bike_crashes[missing_coords_mask]

print(f"Crashes missing coordinates: {len(crashes_missing_coords)}")

# Fill coordinates for crashes that have intersection matches
filled_count = 0
for idx, row in crashes_missing_coords.iterrows():
    intersection_id = row['Intersection_ID']
    if intersection_id in intersection_coords.index:
        bike_crashes.at[idx, 'Latitude'] = intersection_coords.loc[intersection_id, 'Latitude']
        bike_crashes.at[idx, 'Longitude'] = intersection_coords.loc[intersection_id, 'Longitude']
        filled_count += 1

print(f"Coordinates filled using intersection averages: {filled_count}")

# Final coordinate statistics
final_crashes_with_coords = bike_crashes.dropna(subset=['Latitude', 'Longitude'])
print(f"\nFINAL COORDINATE COVERAGE:")
print(f"Total bicycle crashes: {len(bike_crashes):,}")
print(f"Crashes with coordinates: {len(final_crashes_with_coords):,}")
print(f"Percentage with coordinates: {len(final_crashes_with_coords)/len(bike_crashes)*100:.1f}%")


CREATING INTERSECTION IDENTIFIERS:


INFERRING COORDINATES FOR MISSING DATA:
Intersections with known coordinates: 157
Crashes missing coordinates: 726
Coordinates filled using intersection averages: 141

FINAL COORDINATE COVERAGE:
Total bicycle crashes: 1,137
Crashes with coordinates: 552
Percentage with coordinates: 48.5%


In [10]:
# Create final cleaned dataset with only important information
print("CREATING FINAL CLEANED DATASET:")
print("="*40)

# Select only important columns for the final dataset
important_columns = [
    # Basic crash information
    'Date Time',
    'Day of Week',
    
    # Location information (cleaned)
    'Street Name Cleaned',
    'Cross Street Cleaned', 
    'Intersection_ID',
    'Latitude',
    'Longitude',
    
    # Crash characteristics
    'Manner of Collision',
    'First Harmful Event',
    
    # Environmental factors
    'Weather Condition 1',
    'Weather Condition 2',
    'Ambient Light',
    'Road Surface Condition',
    
    # Traffic control
    'Traffic Control Device Type',
    'Traffic Control Device Functionality',
    'Roadway Junction Type',
    'Trafficway Description',
    'Speed Limit',
    
    # Vehicle information
    'Object 1',
    'Object 2',
    
    # Injury information
    'P1 Injury',
    'P2 Injury',
    'P1 Age',
    'P2 Age',
    'P1 Sex',
    'P2 Sex',
    
    # Bicycle-specific indicators
    'May involve cyclist',
    'May Involve Pedestrian',
    'P1 Non Motorist Desc',
    'P2 Non Motorist Desc'
]

# Create the cleaned dataset
cleaned_dataset = bike_crashes[important_columns].copy()

# Convert Date Time to datetime
cleaned_dataset['Date Time'] = pd.to_datetime(cleaned_dataset['Date Time'])

# Add derived features
cleaned_dataset['Year'] = cleaned_dataset['Date Time'].dt.year
cleaned_dataset['Month'] = cleaned_dataset['Date Time'].dt.month
cleaned_dataset['Day'] = cleaned_dataset['Date Time'].dt.day
cleaned_dataset['Hour'] = cleaned_dataset['Date Time'].dt.hour
cleaned_dataset['Day_of_Week_Num'] = cleaned_dataset['Date Time'].dt.dayofweek
cleaned_dataset['Is_Weekend'] = cleaned_dataset['Day_of_Week_Num'].isin([5, 6])
cleaned_dataset['Is_Rush_Hour'] = cleaned_dataset['Hour'].isin([7, 8, 9, 16, 17, 18])

# Create season feature
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

cleaned_dataset['Season'] = cleaned_dataset['Month'].apply(get_season)

# Remove rows without coordinates (as requested)
cleaned_dataset = cleaned_dataset.dropna(subset=['Latitude', 'Longitude'])

print(f"Final cleaned dataset shape: {cleaned_dataset.shape}")
print(f"Records with coordinates: {len(cleaned_dataset):,}")

# Show top intersections in cleaned dataset
print(f"\nTop 10 intersections in cleaned dataset:")
top_intersections = cleaned_dataset['Intersection_ID'].value_counts().head(10)
print(top_intersections)


CREATING FINAL CLEANED DATASET:
Final cleaned dataset shape: (552, 38)
Records with coordinates: 552

Top 10 intersections in cleaned dataset:
Intersection_ID
MASSACHUSETTS AVENUE                      137
CAMBRIDGE STREET                           35
BROADWAY                                   29
HAMPSHIRE STREET                           25
ALBANY STREET_AND_MASSACHUSETTS AVENUE     14
VASSAR ST                                  10
JFK STREET                                  9
PROSPECT STREET                             9
MAIN STREET                                 8
COLUMBIA ST                                 7
Name: count, dtype: int64


In [11]:
# Create the processed data directory and save the cleaned dataset
import os

# Create processed data directory
processed_dir = './data/processed'
os.makedirs(processed_dir, exist_ok=True)

# Save the cleaned dataset
output_file = os.path.join(processed_dir, 'bicycle_crashes_cleaned.csv')
cleaned_dataset.to_csv(output_file, index=False)

print(f"SAVED CLEANED DATASET:")
print("="*30)
print(f"File: {output_file}")
print(f"Records: {len(cleaned_dataset):,}")
print(f"Columns: {len(cleaned_dataset.columns)}")
print(f"File size: {os.path.getsize(output_file) / 1024**2:.2f} MB")

# Show final summary
print(f"\nFINAL SUMMARY:")
print("="*20)
print(f"Original bicycle crashes: {len(bike_crashes):,}")
print(f"Cleaned dataset (with coordinates): {len(cleaned_dataset):,}")
print(f"Data retention rate: {len(cleaned_dataset)/len(bike_crashes)*100:.1f}%")

print(f"\nStreet name standardization results:")
print(f"  - Massachusetts Avenue: {len(cleaned_dataset[cleaned_dataset['Street Name Cleaned'] == 'MASSACHUSETTS AVENUE'])} crashes")
print(f"  - Cambridge Street: {len(cleaned_dataset[cleaned_dataset['Street Name Cleaned'] == 'CAMBRIDGE STREET'])} crashes")
print(f"  - Broadway: {len(cleaned_dataset[cleaned_dataset['Street Name Cleaned'] == 'BROADWAY'])} crashes")

print(f"\nUnique intersections: {cleaned_dataset['Intersection_ID'].nunique()}")
print(f"Date range: {cleaned_dataset['Date Time'].min()} to {cleaned_dataset['Date Time'].max()}")

# Show sample of the cleaned data
print(f"\nSAMPLE OF CLEANED DATA:")
print("="*30)
sample_columns = ['Date Time', 'Street Name Cleaned', 'Cross Street Cleaned', 'Intersection_ID', 
                  'Latitude', 'Longitude', 'Manner of Collision', 'Weather Condition 1']
print(cleaned_dataset[sample_columns].head(10))


SAVED CLEANED DATASET:
File: ./data/processed/bicycle_crashes_cleaned.csv
Records: 552
Columns: 38
File size: 0.45 MB

FINAL SUMMARY:
Original bicycle crashes: 1,137
Cleaned dataset (with coordinates): 552
Data retention rate: 48.5%

Street name standardization results:
  - Massachusetts Avenue: 256 crashes
  - Cambridge Street: 35 crashes
  - Broadway: 29 crashes

Unique intersections: 157
Date range: 2015-01-05 09:17:00 to 2025-08-24 10:40:00

SAMPLE OF CLEANED DATA:
              Date Time   Street Name Cleaned Cross Street Cleaned  \
34  2025-08-24 10:40:00  MASSACHUSETTS AVENUE         BEECH STREET   
38  2025-08-22 16:00:00  MASSACHUSETTS AVENUE         STATE STREET   
69  2025-08-16 11:55:00  MASSACHUSETTS AVENUE    LANDSDOWNE STREET   
70  2025-08-15 21:51:00  MASSACHUSETTS AVENUE                 None   
95  2025-08-11 15:30:00             QUINCY ST                 None   
162 2025-07-26 15:06:00  MASSACHUSETTS AVENUE        MT. VERNON ST   
179 2025-07-23 14:44:00            J